## Install libraries

In [1]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken "<YOUR_NGROK_KEY>"

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=6b7d715b74d8e6572c5aab887fe6f673fe17764df66b40cdb54c981d82f4b920
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install xformers
!pip install bitsandbytes accelerate transformers
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c8088282b752514e2a2e0c44003ffda12696dfbcaeb1d9085e595d41ab44f622
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentenc

## Import libraries

In [3]:
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone as Pc
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys
from dotenv import load_dotenv


load_dotenv()

True

In [4]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

## sentence-transformers embeddings from huggingface

In [5]:
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## loading env vars

In [6]:
PINECONE_INDEX = os.getenv('PINECONE_INDEX')
PINECONE_HOST = os.getenv('PINECONE_HOST')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
HUGGING_FACE_ACCESS_TOKEN = os.getenv('HUGGING_FACE_ACCESS_TOKEN')

## connecting to an existing index

In [7]:
index_name = PINECONE_INDEX
host = PINECONE_HOST
pc = Pc(api_key = PINECONE_API_KEY)
index = pc.Index(index_name, host = PINECONE_HOST)

In [8]:
docsearch = Pinecone(
    index, embeddings.embed_query, "text"
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## load models

In [9]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token = HUGGING_FACE_ACCESS_TOKEN)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
import torch

In [11]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                             use_auth_token = HUGGING_FACE_ACCESS_TOKEN,
                                             load_in_8bit = True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [12]:
pipe = pipeline("text-generation",
                model = model,
                tokenizer = tokenizer,
                torch_dtype = torch.bfloat16,
                device_map = "auto",
                max_new_tokens = 512,
                do_sample = True,
                top_k = 30,
                num_return_sequences = 1,
                eos_token_id = tokenizer.eos_token_id
                )

In [13]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature': 0.1})


In [14]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""


In [15]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [16]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [17]:
instruction = """
{context}

Question: {question}
"""

In [18]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [19]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<</SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [20]:
prompt = PromptTemplate(template = template, input_variables = ["context", "question"])

## Q&A

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = docsearch.as_retriever(search_kwargs = {"k": 2}),
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt},
)

In [32]:
# result = qa_chain("What are the main features of cloud computing program?")

In [37]:
# result['source_documents'][0].metadata['source']

'https://brainlox.com/courses/e0edfcf8-9e0f-4c7a-bc90-3000822924e2'

In [25]:
# f"{result['result']}"

'  Based on the provided context, the main features of the cloud computing program are:\n\n1. One on One Live Interactive Sessions with a verified instructor.\n2. Hands-on skills building using core Amazon Web Services (AWS) services.\n3. Coverage of general cloud computing concepts and AWS from fundamentals to advanced concepts.\n4. Solid understanding of AWS at the end of the course, allowing students to start using it for their projects.'

## flask API

In [22]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return "<h1>Send a POST request to /model</h1>"


@app.route('/model', methods = ['POST'])
def message():
    if request.method == 'POST':
        query = request.args.get('query', '')

        response = qa_chain(query)

        sdocs = response['source_documents']
        src = ''
        for sdoc in sdocs:
          src += sdoc.metadata['source'] + ', '
        src = src.strip(', ')
        return response['result'].strip().replace("\n", " ") + f'\n\nSource(s): {src}'

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://7736-34-125-22-101.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Jan/2024 15:18:40] "POST /?query=What%20did%20Stephen%20Hawking%20say?&query=What%20do%20our%20robots%20today%20do? HTTP/1.1" 405 -
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
INFO:werkzeug:127.0.0.1 - - [22/Jan/2024 15:19:39] "POST /model?query=What%20do%20our%20robots%20today%20do? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2024 15:21:43] "POST /model?query=What%20is%20the%20cost%20per%20session%20for%20chatbot%20building%20course%20(kids)? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2024 15:23:17] "POST /model?query=What%20did%20Stephen%20Hawking%20say? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2024 15:24:36] "POST /model?query=Building%20AI%20Chatbot%20Like%20Chatgpt%20(For%20Kids)%20day%2010? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - 